<a href="https://colab.research.google.com/github/bdi2357/QnA_examine/blob/main/L5_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[link text](https://)# LangChain: Evaluation

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [ ]:
!git clone https://github.com/openai/openai-quickstart-node.git
!pip install openai
!pip install tiktoken
!pip install python-dotenv
import os
os.chdir("openai-quickstart-node")
!cp .env.example .env
import os

file_path = '.env'  # Update with the actual path to the .env file
api_key =   # Replace with your new OpenAI API key

line_to_replace = 'OPENAI_API_KEY='

with open(file_path, 'r+') as f:
    lines = f.readlines()
    f.seek(0)  # Reset the file pointer to the beginning

    for line in lines:
        if line.startswith(line_to_replace):
            f.write(f'{line_to_replace}{api_key}\n')
        else:
            f.write(line)

    f.truncate()  # Remove any remaining content after the updated line

Cloning into 'openai-quickstart-node'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 120 (delta 40), reused 32 (delta 28), pack-reused 61
Receiving objects: 100% (120/120), 93.61 KiB | 5.20 MiB/s, done.
Resolving deltas: 100% (52/52), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.1 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp ../drive/MyDrive/OutdoorClothingCatalog_1000.csv .

In [ ]:
import os
import openai
import sys
sys.path.append('../..')
#import utils

import panel as pn  # GUI
pn.extension()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
!pip install docarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.2/263.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 5.0 MB/s eta 0:00:00


## Create our QandA application

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
RetrievalQA.from_chain_type?

In [ ]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Coming up with test datapoints

In [ ]:
data[10]

Document(page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10})

In [ ]:
data[11]

Document(page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 11})

### Hard-coded examples

In [ ]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

In [ ]:
My_Example = [
     {
        "query": "Do you have Cozy Comfort Pullover Set without side pockets?"
    },
     {
        "query" : "Do you have Gorilla pants for men"
     },
     {"query": "Is there a collection with Down hooded Jacket?"},
     {"query": "Is there a collection with Down hooded Jacket? Do you think that items in this collection could fit a 10 YO boy ?"},
      {"query": "Are there outfits that could fit a 9 YO boy about 1.4 meters height 34 kg  weight?"},
     {"query": "Are there outfits that could fit a 11 YO boy about 1.52 meters height 45 kg  weight, median waist size as well as other measures are normal with respect to the age height and weight?"}


]

### LLM-Generated examples

In [ ]:
from langchain.evaluation.qa import QAGenerateChain


In [ ]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [ ]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:303: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [ ]:
new_examples[0]

{'qa_pairs': {'query': "What is the weight of each pair of Women's Campside Oxfords?",
  'answer': "The weight of each pair of Women's Campside Oxfords is approximately 1 lb. 1 oz."}}

In [ ]:
data[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

### Combine examples

In [ ]:
examples += new_examples

In [ ]:
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


'Yes, the Cozy Comfort Pullover Set does have side pockets.'

In [ ]:
for ii in range(len(My_Example)):
  print(qa.run(My_Example[ii]["query"]))



> Entering new RetrievalQA chain...

> Finished chain.
I'm sorry, but I don't have that information.


> Entering new RetrievalQA chain...

> Finished chain.
No, I do not have any information about Gorilla pants for men.


> Entering new RetrievalQA chain...

> Finished chain.
Yes, there is a collection with a Down hooded jacket. The Ultra-Lofty 850 Stretch Down Hooded Jacket is part of the DownTek collection.


> Entering new RetrievalQA chain...

> Finished chain.
Yes, there is a collection with a Down hooded Jacket. The Ultra-Lofty 850 Stretch Down Hooded Jacket is part of the DownTek collection. As for whether the items in this collection could fit a 10-year-old boy, I don't have that information.


> Entering new RetrievalQA chain...

> Finished chain.
Based on the information provided, it is difficult to determine if there are specific outfits that would fit a 9-year-old boy who is approximately 1.4 meters tall and weighs 34 kg. It would be best to refer to the specific size ch

## Manual Evaluation

In [ ]:
import langchain
langchain.debug = True

In [ ]:
qa.run(examples[0]["query"])

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded h

'Yes, the Cozy Comfort Pullover Set does have side pockets.'

In [ ]:
# Turn off the debug mode
langchain.debug = False

In [ ]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'qa_pairs': {'query': "What is the weight of each pair of Women's Campside Oxfords?",
   'answer': "The weight of each pair of Women's Campside Oxfords is approximately 1 lb. 1 oz."}},
 {'qa_pairs': {'query': 'What are the dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave?',
   'answer': 'The dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave are 18" x 28".'}},
 {'qa_pairs': {'query': "What are some features of the Infant and Toddler Girls' Coastal Chill Swimsuit?",
   'answer': "The swimsuit features bright colors, ruffles, and exclusive whimsical prints. It is made of four-way-stretch and chlorine-resistant fabric that keeps its shape and resists snags. The fabric is also UPF 50+ rated, providing the highest

## LLM assisted evaluation

In [ ]:
examples_old = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

In [ ]:
predictions_old = qa.apply(examples_old)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [ ]:
new_examples

[{'qa_pairs': {'query': "What is the weight of each pair of Women's Campside Oxfords?",
   'answer': "The weight of each pair of Women's Campside Oxfords is approximately 1 lb. 1 oz."}},
 {'qa_pairs': {'query': 'What are the dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave?',
   'answer': 'The dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave are 18" x 28".'}},
 {'qa_pairs': {'query': "What are some features of the Infant and Toddler Girls' Coastal Chill Swimsuit?",
   'answer': "The swimsuit features bright colors, ruffles, and exclusive whimsical prints. It is made of four-way-stretch and chlorine-resistant fabric that keeps its shape and resists snags. The fabric is also UPF 50+ rated, providing the highest rated sun protection possible by blocking 98% of the sun's harmful rays. The swimsuit has crossover no-slip straps and a fully lined bottom for a secure fit and maximum coverage. It can be machine washed and line dried for be

In [ ]:
predictions_new = qa.apply([e['qa_pairs'] for e in new_examples])



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [ ]:
new_examples_q = [e['qa_pairs'] for e in new_examples]

In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
graded_outputs = eval_chain.evaluate(examples_old, predictions_old)

In [ ]:
for i, eg in enumerate(examples_old):
    print(f"Example {i}:")
    print("Question: " + predictions_old[i]['query'])
    print("Real Answer: " + predictions_old[i]['answer'])
    print("Predicted Answer: " + predictions_old[i]['result'])
    print("Predicted Grade: ",graded_outputs[i])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set does have side pockets.
Predicted Grade:  {'results': 'CORRECT'}

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade:  {'results': 'CORRECT'}



In [ ]:
graded_outputs_new = eval_chain.evaluate(new_examples_q, predictions_new)

In [ ]:
for i, eg in enumerate(new_examples_q):
    print(f"Example {i}:")
    print("Question: " + predictions_new[i]['query'])
    print("Real Answer: " + predictions_new[i]['answer'])
    print("new_examples ans:" + new_examples[i]['qa_pairs']['answer'])
    print("Predicted Answer: " + predictions_new[i]['result'])
    print("Predicted Grade: ",graded_outputs_new[i])
    print()

Example 0:
Question: What is the weight of each pair of Women's Campside Oxfords?
Real Answer: The weight of each pair of Women's Campside Oxfords is approximately 1 lb. 1 oz.
new_examples ans:The weight of each pair of Women's Campside Oxfords is approximately 1 lb. 1 oz.
Predicted Answer: The weight of each pair of Women's Campside Oxfords is approximately 1 lb. 1 oz.
Predicted Grade:  {'results': 'CORRECT'}

Example 1:
Question: What are the dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave?
Real Answer: The dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave are 18" x 28".
new_examples ans:The dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave are 18" x 28".
Predicted Answer: The dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave are 18" x 28".
Predicted Grade:  {'results': 'CORRECT'}

Example 2:
Question: What are some features of the Infant and Toddler Girls' Coastal Chill Swim

In [ ]:
[x for x in dir(eval_chain) if  x[:1] != "_"]

['Config',
 'aapply',
 'aapply_and_parse',
 'abatch',
 'acall',
 'aevaluate_strings',
 'agenerate',
 'ainvoke',
 'apply',
 'apply_and_parse',
 'apredict',
 'apredict_and_parse',
 'aprep_prompts',
 'arun',
 'astream',
 'batch',
 'bind',
 'callback_manager',
 'callbacks',
 'construct',
 'copy',
 'create_outputs',
 'dict',
 'evaluate',
 'evaluate_strings',
 'evaluation_name',
 'from_llm',
 'from_orm',
 'from_string',
 'generate',
 'input_keys',
 'invoke',
 'json',
 'lc_attributes',
 'lc_namespace',
 'lc_secrets',
 'lc_serializable',
 'llm',
 'llm_kwargs',
 'memory',
 'metadata',
 'output_key',
 'output_keys',
 'output_parser',
 'parse_file',
 'parse_obj',
 'parse_raw',
 'predict',
 'predict_and_parse',
 'prep_inputs',
 'prep_outputs',
 'prep_prompts',
 'prompt',
 'raise_callback_manager_deprecation',
 'requires_input',
 'requires_reference',
 'return_final_only',
 'run',
 'save',
 'schema',
 'schema_json',
 'set_verbose',
 'stream',
 'tags',
 'to_json',
 'to_json_not_implemented',
 'updat